## Teoria e Implementazione degli Embeddings: Node2Vec su CPU vs GPU

Questo documento esplora il funzionamento teorico degli embeddings su grafo (specificamente Node2Vec) e analizza come l'architettura hardware (CPU vs GPU) influenzi drasticamente l'approccio computazionale, specialmente per grafi di grandi dimensioni (350k nodi).


### Il Concetto Teorico: Cos'è un Graph Embedding?
L'obiettivo di un embedding è tradurre la struttura complessa e topologica di un grafo in uno spazio vettoriale semplice (spazio Euclideo). Dato un grafo $G = (V, E)$, vogliamo apprendere una funzione $f: V \rightarrow \mathbb{R}^d$ (dove $d$ è la dimensione, es. 128) tale che:

* Se due nodi $u$ e $v$ sono "simili" nel grafo, i loro vettori $z_u$ e $z_v$ devono essere vicini nello spazio vettoriale.

La "vicinanza" vettoriale è solitamente misurata tramite il prodotto scalare.

#### L'Intuizione di Node2Vec
Node2Vec si basa su un'analogia linguistica derivata da Word2Vec (NLP):
* In un testo, le parole che appaiono nello stesso contesto sono semanticamente simili.
* In un grafo, i nodi che appaiono nella stessa "passeggiata" (path) sono strutturalmente simili.

L'algoritmo si divide in due fasi principali:
1. Campionamento (Random Walks): Generazione di sequenze di nodi simulate attraversando il grafo.
2. Ottimizzazione (Skip-Gram): Addestramento di una rete neurale per predire i nodi vicini dato un nodo di input.

### Fase 1: Random Walks (Il Collo di Bottiglia)
Node2Vec introduce una passeggiata casuale parametrica del secondo ordine. Non si muove puramente a caso, ma è guidata da due parametri:
* $p$ (Return parameter): Probabilità di tornare al nodo precedente (favorisce la struttura locale, simile a BFS).
* $q$ (In-out parameter): Probabilità di allontanarsi (favorisce l'esplorazione globale, simile a DFS).

#### Implementazione CPU (Libreria node2vec)
* Logica: La CPU pre-calcola le probabilità di transizione per ogni nodo (o le calcola al volo) e simula sequenzialmente le camminate.
* Il Problema: Python è un linguaggio interpretato e single-threaded (per via del GIL). Anche usando il multiprocessing (workers=8), la generazione di camminate è un processo seriale per ogni core:
    1. Sono al nodo A.
    2. Guardo i vicini.
    3. Estraggo un numero casuale.
    4. Scelgo il nodo B.
    5. Ripeto 80 volte.
* Effetto su 350k nodi: Devi generare $350.000 \times 10 = 3.500.000$ camminate. Questo richiede milioni di operazioni di accesso alla memoria e generazione di numeri casuali, saturando la cache della CPU.

#### Implementazione GPU (PyTorch Geometric)
* Logica: La GPU eccelle nel parallelismo SIMD (Single Instruction, Multiple Data). Invece di simulare una camminata alla volta, la GPU gestisce un batch di migliaia di camminate simultaneamente.
* Vettorializzazione: PyTorch Geometric utilizza operazioni su tensori sparsi. La matrice di adiacenza è caricata in VRAM. Il campionamento del prossimo passo per 10.000 camminate avviene in un unico "colpo" di clock (o quasi) grazie ai CUDA cores.

### Fase 2: Ottimizzazione (Skip-Gram con Negative Sampling)
Una volta ottenute le sequenze (es. [A, B, C, D, ...]), l'obiettivo è massimizzare la probabilità di vedere il contesto (es. B, C) dato l'input (A).
La funzione obiettivo (Loss) cerca di massimizzare il prodotto scalare tra nodi vicini e minimizzarlo tra nodi non connessi:$$J(\theta) = - \sum_{(u,v) \in \text{walks}} \log(\sigma(z_u \cdot z_v)) - \sum_{(u,k) \in \text{noise}} \log(1 - \sigma(z_u \cdot z_k))$$

#### Implementazione CPU (gensim Word2Vec)
* Struttura: Usa gensim, che ha un backend ottimizzato in C.
* Processo: Prende le camminate generate (salvate in RAM come liste di stringhe) e addestra il modello.
* Limiti:
    * Data Transfer: Le camminate devono essere passate da Python a C.
    * Memoria: Mantenere 3.5 milioni di camminate in memoria (liste Python) occupa molta RAM.
    * Calcolo: L'aggiornamento dei pesi avviene sulla CPU. Sebbene veloce, non può competere con le operazioni matriciali di una GPU.

#### Implementazione GPU (PyTorch SparseAdam)
* Struttura: Tutto avviene in un ciclo di training PyTorch.
* Processo Dinamico (On-the-fly): A differenza della CPU, PyG spesso genera le camminate dentro il DataLoader ad ogni epoca o batch. Non c'è bisogno di pre-calcolare e salvare 3.5 milioni di sequenze in RAM.
* Negative Sampling Efficiente: Per ogni coppia positiva (nodo reale nel cammino), la GPU estrae molto velocemente $k$ nodi casuali (negativi) e calcola il prodotto scalare tramite moltiplicazione di matrici massiva.
* Sparse Updates: Poiché modifichiamo solo i vettori dei nodi coinvolti nel batch (pochi rispetto a 350k), si usa SparseAdam, un ottimizzatore che aggiorna solo le righe specifiche della matrice degli embeddings, risparmiando tempo di calcolo enorme.

### Riassunto Comparativo
| Caratteristica | Variante CPU (node2vec standard) | Variante GPU (PyTorch Geometric) |
|-----------|-----------|-----------|
| Generazione Walks  | Pre-elaborazione: Lenta, sequenziale, usa molta RAM per salvare le liste.  | On-the-fly: Veloce, parallela, basso consumo RAM (generata e consumata subito).  |
| Gestione Memoria  | Richiede RAM di sistema per il grafo + tutte le camminate.  | Richiede VRAM (8GB) per il grafo e i batch correnti. Molto più efficiente.  |
| Addestramento  | Word2Vec (Gensim/C). Ottimizzato ma limitato dai core CPU.  | SGD/Adam su Tensori CUDA. Parallelismo massivo.  |
| Scalabilità  | Lineare rispetto al numero di nodi (molto lento sopra 100k nodi).  | Sub-lineare (grazie al batching massivo), scala benissimo su milioni di nodi.  |



### Conclusione per il tuo caso
Con 350k nodi e 1.1M archi, ti trovi esattamente nel punto in cui l'approccio CPU diventa frustrante (ore di attesa) e l'approccio GPU brilla (minuti). La tua RTX 4060 è perfetta per gestire la matrice sparsa di queste dimensioni, permettendo al DataLoader di PyTorch di saturare la banda passante della scheda video piuttosto che aspettare i calcoli sequenziali della CPU.

In [1]:
import networkx as nx
import pickle

graph_path = "../data/processed/amazon_graph.pickle"
with open(graph_path, "rb") as f:
    G = pickle.load(f)

### CPU version

In [ ]:
import time
import os
from node2vec import Node2Vec

In [ ]:
def get_node2vec_emb_CPU(G, embedding_dim=128, walk_length=80, window=10,
                         walks_per_node=10, p=1, q=2,):

    """
    Computes Node2Vec embeddings on the CPU using the standard `node2vec` library.

    Args:
        G (nx.Graph): Input NetworkX graph.
        embedding_dim (int): Dimension of the output embedding vectors.
        walk_length (int): Length of each random walk.
        window (int): Window size for the skip-gram model.
        walks_per_node (int): Number of random walks generated per node.
        p (float): Return parameter (likelihood of returning to the immediate source).
        q (float): In-out parameter (likelihood of moving away from the source).

    Returns:
        pd.DataFrame: DataFrame containing 'ASIN' (original ID) and embedding columns.
    """

    print("Starting embeddings computation on CPU using node2vec library")
    start_time = time.time()

    # Model configuration
    node2vec_model = Node2Vec(
        G,
        dimensions=embedding_dim,
        walk_length=walk_length,
        num_walks=walks_per_node,
        workers=os.cpu_count(),
        p=p,
        q=q,
        quiet=False
    )

    # Model training
    # window: max nodes distance at which the algorith will try to predict relations
    # min_count: will consider also nodes that appear only 1 time
    model = node2vec_model.fit(window = window, min_count = 1, batch_words = 4)

    end_time = time.time()
    print(f"Tempo totale CPU: {end_time - start_time:.2f} secondi")

    # Output DataFrame building
    df = pd.DataFrame(
        index=model.wv.index_to_key,
        data=model.wv.vectors
    )

    df.columns = [f"emb_{i}" for i in range(model.vector_size)]
    df = df.reset_index() # push the index (ASIN) to be a standard column
    df = df.rename(columns={'index': 'ASIN'}) # rename the "index" column

    return df

### GPU version

In [2]:
import time
import os
import torch
from torch_geometric.nn import Node2Vec as torch_Node2Vec
from torch_geometric.utils import from_networkx
import pandas as pd

In [3]:
def get_node2vec_emb_GPU(G, embedding_dim=128,
                         walk_length=80, context_size=10, walks_per_node=10,
                         p=1, q=2, epochs=100, patience=3, batch_size=128):
    """
    Computes Node2Vec embeddings using GPU acceleration via PyTorch Geometric.

    This function preprocesses the input graph (attribute clearing and integer relabeling),
    trains the model using an Early Stopping mechanism and re-maps the resulting vectors to
    the original node IDs.

    Args:
        G (nx.Graph): Input NetworkX graph.
        embedding_dim (int): Dimension of the output embedding vectors.
        walk_length (int): Length of each random walk.
        context_size (int): Window size for the skip-gram model.
        walks_per_node (int): Number of random walks generated per node.
        p (float): Return parameter (likelihood of returning to the immediate source).
        q (float): In-out parameter (likelihood of moving away from the source).
        epochs (int): Maximum number of training epochs.
        patience (int): Epochs to wait for loss improvement before early stopping.
        batch_size (int): Number of nodes per training batch.

    Returns:
        pd.DataFrame: A DataFrame containing the 'ASIN' (original node ID) and the
                      corresponding embedding vectors.
    """

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    if device == 'cuda':
        print("CUDA available, using PyTorch on GPU.")
    if device == 'cpu':
        print("CUDA not available, using PyTorch on CPU.")

    start_time = time.time()

    # Remove attributes from the graph's nodes
    G_clean = G.copy()
    for n in G_clean.nodes():       # for each node's ID n in G_clean...
        G_clean.nodes[n].clear()    # access to the attributes dictionary

    # Create a graph copy with integer node's labels
    nodes_list = list(G_clean.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes_list)}
    reverse_mapping = {i: node for i, node in enumerate(nodes_list)}
    G_int = nx.relabel_nodes(G_clean, node_mapping)

    # Convert the graph to PyTorch Geometric's Data object
    data = from_networkx(G_int)
    data = data.to(device)

    # Model configuration
    model = torch_Node2Vec(
        data.edge_index,
        embedding_dim=embedding_dim,
        walk_length=walk_length,
        context_size=context_size,
        walks_per_node=walks_per_node,
        num_negative_samples=1,
        p=p,
        q=q,
        sparse=True
    ).to(device)

    # Generates walks on-the-fly and set batches of 128 nodes
    loader = model.loader(batch_size=batch_size, shuffle=True, num_workers=0)

    # set the appropriate optimizer since the matrix is sparse
    optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

    # Training
    model.train()               # start training mode
    best_loss = float('inf')    # + infinity
    counter = 0

    print(f"Starting training")
    for epoch in range(epochs):
        total_loss = 0

        # for each nodes batch, read the pair of positive random walks and negative random walks
        for pos_rw, neg_rw in loader:
            optimizer.zero_grad()
            loss = model.loss(pos_rw.to(device), neg_rw.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch: {epoch+1:02d}, Loss: {total_loss / len(loader):.4f}")

        # Stopping logic
        avg_loss = total_loss / len(loader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            counter = 0  # reset the counter since
        else:
            counter += 1
            print(f"   No improvement detected for {counter} epochs.")

        if counter >= patience:
            print(f"Stopping model training since there has been no improvement for {patience} epochs.")
            break

    end_time = time.time()
    print(f"Total GPU time: {end_time - start_time:.2f} s")

    # Embeddings extraction
    model.eval()                    # stop training mode
    with torch.no_grad():           # do not compute gradients
        z = model().cpu().numpy()   # bring values to CPU

    # Remap list index to the original node ID
    emb_dict = {nodes_list[i]: z[i] for i in range(len(nodes_list))}
    df = pd.DataFrame.from_dict(emb_dict, orient='index')

    df.columns = [f"emb_{i}" for i in range(df.shape[1])]
    df = df.reset_index() # push the index (ASIN) to be a standard column
    df = df.rename(columns={'index': 'ASIN'}) # rename the "index" column

    return df


In [4]:
if __name__ == "__main__":

    embeddings = get_node2vec_emb_GPU(G)

    print(f"DataFrame dimensions: {embeddings.shape}")
    print(embeddings.head())

    embeddings.to_csv(
        '../data/processed/embeddings_p1_q2.csv',
        index=False,         # Row index is not needed
        float_format='%.6f',
        chunksize=10000
    )

CUDA available, using PyTorch on GPU.
Starting training
Epoch: 01, Loss: 1.8085
Epoch: 02, Loss: 0.7733
Epoch: 03, Loss: 0.7409
Epoch: 04, Loss: 0.7287
Epoch: 05, Loss: 0.7223
Epoch: 06, Loss: 0.7189
Epoch: 07, Loss: 0.7171
Epoch: 08, Loss: 0.7162
Epoch: 09, Loss: 0.7160
Epoch: 10, Loss: 0.7159
Epoch: 11, Loss: 0.7159
   No improvement detected for 1 epochs.
Epoch: 12, Loss: 0.7160
   No improvement detected for 2 epochs.
Epoch: 13, Loss: 0.7160
   No improvement detected for 3 epochs.
Epoch: 14, Loss: 0.7160
   No improvement detected for 4 epochs.
Epoch: 15, Loss: 0.7160
   No improvement detected for 5 epochs.
Stopping model training since there has been no improvement for 5 epochs.
Total GPU time: 4331.63 s
DataFrame dimensions: (334843, 129)
         ASIN     emb_0     emb_1     emb_2     emb_3     emb_4     emb_5  \
0  0827229534 -0.039401  0.319984  0.147284 -0.009532  0.272835 -0.001497   
1  0738700797 -0.084367 -0.033158  0.071312  0.463542  0.294667  0.372568   
2  084232832